# run

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision as tv
import math

train_dataset = tv.datasets.CIFAR10(root="./", train=True,
transform=tv.transforms.ToTensor(),
download=True)
test_dataset = tv.datasets.CIFAR10(root="./", train=False,
transform=tv.transforms.ToTensor(),
download=True)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                 batch_size=100,
                                 shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                 batch_size=100,
                                 shuffle=False)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [13]:
MODEL_NAME2 = 'CIFAR10_2D.model'
EPOCH = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
class CIFAR10_2D(torch.nn.Module):
    def __init__(self):
        super(CIFAR10_2D, self).__init__()
        num_classes=10

        self.block1_output = nn.Sequential (
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block2_output = nn.Sequential (
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block3_output = nn.Sequential (
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block4_output = nn.Sequential (
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.block5_output = nn.Sequential (
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(512, 512), 
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(512, 32 ),  
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(32, num_classes),  
        )
    def forward(self, x):
        x = self.block1_output(x)
        x = self.block2_output(x)
        x = self.block3_output(x)
        x = self.block4_output(x)
        x = self.block5_output(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

In [4]:
def train():
    model = CIFAR10_2D().to(DEVICE)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
    for epoch in range(EPOCH):
        loss =0 
        for images, labels in train_loader:
            images = images.view(-1,3,32,32).to(DEVICE)
            labels = labels.to(DEVICE)
            optimizer.zero_grad()
            y = model(images)
            batchloss =F.cross_entropy(y, labels)
            batchloss.backward()
            optimizer.step()
            loss = loss + batchloss.item()
        print('epoch',epoch,': loss',loss)
    torch.save(model.state_dict(),MODEL_NAME2)

In [14]:
def test():
    total = len(test_loader.dataset)
    correct = 0
    model = CIFAR10_2D().to('cpu')
    model.load_state_dict(torch.load(MODEL_NAME2))
    model.eval()
    for images, labels in test_loader:
        images = images.view(-1,3,32,32).to('cpu')
        y = model(images)
        pred_labels = y.max(dim=1)[1]
        correct = correct + (pred_labels ==labels).sum()
    print('correct:',correct.item())
    print('total:',total)
    print('accuracy:',(correct.item()/float(total)))

In [6]:
train()

epoch 0 : loss 819.7196246385574
epoch 1 : loss 569.868083178997
epoch 2 : loss 452.54128724336624
epoch 3 : loss 375.78697443008423
epoch 4 : loss 319.688482940197
epoch 5 : loss 271.6364771425724
epoch 6 : loss 236.69614379107952
epoch 7 : loss 199.17387522757053
epoch 8 : loss 176.73684014379978
epoch 9 : loss 157.48312655091286


In [15]:
test()

correct: 8032
total: 10000
accuracy: 0.8032
